In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from langdetect import detect
plt.style.use('seaborn')

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', -1)

In [ ]:
l_data=pd.read_csv('/content/cleaned_melbourne_listings v1.1.csv')
r_data=pd.read_csv('/content/reviews_dec18 2.csv')

In [ ]:
print("The dataset has {} rows and {} columns.".format(*l_data.shape))


The dataset has 17146 rows and 15 columns.


In [ ]:
print("The dataset has {} rows and {} columns.".format(*r_data.shape))


The dataset has 320856 rows and 6 columns.


In [ ]:
df = pd.merge(l_data,r_data,left_on='listing_id',right_on='listing_id',how='left')


In [ ]:
df.head(1)


,listing_id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,id,date,reviewer_id,reviewer_name,comments
0,2449576,"Southbank Warehouse, Free Parking & Wifi",3373049,Terry,Melbourne,-37.828398,144.959993,Entire home/apt,109,1,479,02-12-2018,10.32,1,259,26580022.0,2015-02-15,12312073.0,Nina,"A fantastic experience for our first Air BnB adventure. Terry was lovely and extremely hospitable. The apartment was perfect - everything we needed and more. Very comfortable and the breakfast provisions, fresh juice and berries were a lovely touch. She also provided info on the local area, festivals, current maps and recommendations for coffee/breaky. The location was great - so close to the CBD, Southbank and South Melbourne and with the garage provided we didn't have to worry about our car for our entire stay. This space would be perfect for either a short or long term stay. We will definitely be back and I highly recommend Terry's groovy warehouse apartment."


In [ ]:
df.drop(['id'],axis=1,inplace=True)


In [ ]:
df.columns


Index(['listing_id', 'name', 'host_id', 'host_name', 'neighbourhood',
       'latitude', 'longitude', 'room_type', 'price', 'minimum_nights',
       'number_of_reviews', 'last_review', 'reviews_per_month',
       'calculated_host_listings_count', 'availability_365', 'date',
       'reviewer_id', 'reviewer_name', 'comments'],
      dtype='object')

In [ ]:
print("The dataset has {} rows and {} columns.".format(*df.shape))


The dataset has 266705 rows and 19 columns.


In [ ]:
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [ ]:
missing= missing_values_table(df)
missing

Your selected dataframe has 19 columns.
There are 8 columns that have missing values.


,Missing Values,% of Total Values
comments,12040,4.5
reviewer_name,11754,4.4
date,11752,4.4
reviewer_id,11752,4.4
last_review,3818,1.4
reviews_per_month,3818,1.4
host_name,4,0.0
name,3,0.0


In [ ]:
df.dropna(inplace=True)


In [ ]:
df.shape


(254661, 19)

In [ ]:
missing= missing_values_table(df)
missing

Your selected dataframe has 19 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [ ]:
nltk.download('vader_lexicon')


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
!pip install langdetect


In [ ]:
df.describe()

,listing_id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,reviewer_id
count,2.546610e+05,2.546610e+05,254661.000000,254661.000000,254661.000000,254661.000000,254661.000000,254661.000000,254661.000000,254661.000000,2.546610e+05
mean,8.897536e+06,2.758415e+07,-37.817513,145.020247,139.012562,2.240520,122.464362,3.231211,9.627073,184.170081,6.081852e+07
std,5.030896e+06,2.656174e+07,0.059528,0.155623,97.380970,7.692637,85.915040,2.071126,20.957937,126.445673,5.235913e+07
min,4.341400e+04,9.082000e+03,-38.192113,144.544393,14.000000,1.000000,1.000000,0.020000,1.000000,0.000000,2.620000e+02
25%,4.710172e+06,6.679534e+06,-37.825528,144.956258,90.000000,1.000000,57.000000,1.560000,1.000000,63.000000,1.901912e+07
50%,9.034313e+06,2.024099e+07,-37.815414,144.968727,120.000000,2.000000,107.000000,2.980000,2.000000,169.000000,4.541834e+07
75%,1.359144e+07,4.167872e+07,-37.805234,144.997469,165.000000,2.000000,168.000000,4.560000,5.000000,312.000000,9.456972e+07
max,1.682019e+07,1.117212e+08,-37.536264,145.831408,8000.000000,900.000000,538.000000,11.510000,98.000000,365.000000,2.287429e+08


In [ ]:
df.to_csv('final_dataset.csv')

In [ ]:
df.describe()

,listing_id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,reviewer_id
count,2.546610e+05,2.546610e+05,254661.000000,254661.000000,254661.000000,254661.000000,254661.000000,254661.000000,254661.000000,254661.000000,2.546610e+05
mean,8.897536e+06,2.758415e+07,-37.817513,145.020247,139.012562,2.240520,122.464362,3.231211,9.627073,184.170081,6.081852e+07
std,5.030896e+06,2.656174e+07,0.059528,0.155623,97.380970,7.692637,85.915040,2.071126,20.957937,126.445673,5.235913e+07
min,4.341400e+04,9.082000e+03,-38.192113,144.544393,14.000000,1.000000,1.000000,0.020000,1.000000,0.000000,2.620000e+02
25%,4.710172e+06,6.679534e+06,-37.825528,144.956258,90.000000,1.000000,57.000000,1.560000,1.000000,63.000000,1.901912e+07
50%,9.034313e+06,2.024099e+07,-37.815414,144.968727,120.000000,2.000000,107.000000,2.980000,2.000000,169.000000,4.541834e+07
75%,1.359144e+07,4.167872e+07,-37.805234,144.997469,165.000000,2.000000,168.000000,4.560000,5.000000,312.000000,9.456972e+07
max,1.682019e+07,1.117212e+08,-37.536264,145.831408,8000.000000,900.000000,538.000000,11.510000,98.000000,365.000000,2.287429e+08


In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

df['polarity_value']="Default"
df['neg']=0.0
df['pos']=0.0
df['neu']=0.0
df['compound']=0.0
for index,row in df.iterrows():
    ss = sid.polarity_scores(row['comments'])
    df.at[index,'polarity_value'] = ss
    df.at[index,'neg'] = ss['neg']
    df.at[index,'pos'] = ss['pos']
    df.at[index,'neu']= ss['neu']
    df.at[index,'compound'] = ss['compound']
df.head()

,listing_id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,date,reviewer_id,reviewer_name,comments,polarity_value,neg,pos,neu,compound
0,2449576,"Southbank Warehouse, Free Parking & Wifi",3373049,Terry,Melbourne,-37.828398,144.959993,Entire home/apt,109,1,479,02-12-2018,10.32,1,259,2015-02-15,12312073.0,Nina,"A fantastic experience for our first Air BnB adventure. Terry was lovely and extremely hospitable. The apartment was perfect - everything we needed and more. Very comfortable and the breakfast provisions, fresh juice and berries were a lovely touch. She also provided info on the local area, festivals, current maps and recommendations for coffee/breaky. The location was great - so close to the CBD, Southbank and South Melbourne and with the garage provided we didn't have to worry about our car for our entire stay. This space would be perfect for either a short or long term stay. We will definitely be back and I highly recommend Terry's groovy warehouse apartment.","{'neg': 0.0, 'neu': 0.689, 'pos': 0.311, 'compound': 0.9909}",0.000,0.311,0.689,0.9909
1,2449576,"Southbank Warehouse, Free Parking & Wifi",3373049,Terry,Melbourne,-37.828398,144.959993,Entire home/apt,109,1,479,02-12-2018,10.32,1,259,2015-02-22,27229595.0,Dirk,Terry war eine außerordentlich nette und unkomplizierte Gastgeberin. Wir wurden von der Southern Cross Station abgeholt und es wurde uns noch die nähere Umgebung gezeigt. Das Apartment liegt sehr günstig für die Erkundung der City und der South Bank. Es ist geräumig. Nachteil: Das einzige Fenster lässt sich nicht öffnen. Eine Kochmöglichkeit fehlt. Ansonsten ist die Küchenzeile git ausgestattet. Die Klimaanlage ist lediglich ein Innenkühlgerät. Ein großer Supermarkt ist zu Fuß zu erreichen. Insgesamt ist das Preis/Leistungsverhältnis super.,"{'neg': 0.204, 'neu': 0.755, 'pos': 0.041, 'compound': -0.9485}",0.204,0.041,0.755,-0.9485
2,2449576,"Southbank Warehouse, Free Parking & Wifi",3373049,Terry,Melbourne,-37.828398,144.959993,Entire home/apt,109,1,479,02-12-2018,10.32,1,259,2015-02-26,27769507.0,Liwen,"This warehouse apartment is tastefully furnished with bright colours and quirky items from around the world, exactly what Melbourne is all about - which is full of character and never dull. The location is not too far from the city centre, 20 mins walk or just 5 mins by tram. And it is quiet and safe. We felt very comfortable staying here. Terry and David were also very nice people and they have clearly made the effort to ensure we were comfortable by providing a wide selection soap, coffee, tea and cereal, as well as guidebooks and maps of Melbourne. Definitely a great place to stay!","{'neg': 0.0, 'neu': 0.735, 'pos': 0.265, 'compound': 0.9846}",0.000,0.265,0.735,0.9846
3,2449576,"Southbank Warehouse, Free Parking & Wifi",3373049,Terry,Melbourne,-37.828398,144.959993,Entire home/apt,109,1,479,02-12-2018,10.32,1,259,2015-02-28,28072698.0,John,This was my first Air BnB experience and wow what a great place to stay. Terry makes you feel at home right from the moment you meet her. The room is massive with an attached bathroom. Terry provides breakfast cereals/tea/coffee and juices along with wifi. \r\nI really could not fault this place and to top it all off it is only 5 minutes walk to Crown Casino/the tram/the river. It is also only 5 min walk to South Melbourne markets. I will happily recommend this place to my friends and look forward to visiting again. Great host/great room/great location all make for a great stay in Melbourne.,"{'neg': 0.0, 'neu': 0.756, 'pos': 0.244, 'compound': 0.9828}",0.000,0.244,0.756,0.9828
4,2449576,"Southbank Warehouse, Free Parking & Wifi",3373049,Terry,Melbourne,-37.828398,144.959993,Entire home/apt,109,1,479,02-12-2018,10.32,1,259,2015-03-01,27034089.0,David,Terry is very organised and takes pride in her apartment. Great location and good v

In [ ]:
df.to_csv('final_df.csv')